# Citeseer Example

## Data

In [3]:
import json
import numpy as np
import torch
import scipy.sparse as sparse
from dgl.data import CiteseerGraphDataset
dataset = CiteseerGraphDataset()
graph = dataset[0]

DGL backend not selected or invalid.  Assuming PyTorch for now.


/usr/local/lib/python3.7/dist-packages/dgl/data/citation_graph.py:284: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Extracting file to /root/.dgl/citeseer


/usr/local/lib/python3.7/dist-packages/dgl/data/citation_graph.py:284: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


Finished data loading and preprocessing.
  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


We can save the data file in only one file `citeseer.npz`.

In [4]:
# citeseer_feat
node_feats = sparse.csr_matrix(graph.ndata["feat"].numpy())
# citeseer_class
node_class = graph.ndata["label"].numpy()  # (3327,)
# citeseer_edge
edge = torch.stack(graph.edges()).numpy().T
# citeseer only has 1 single graph, there are some isolated nodes in the graph. 
# These isolated nodes are added as zero-vecs into the right position
node_list = np.ones((1, graph.num_nodes()))  # (1, 3327)
edge_list = np.ones((1, graph.num_edges()))  # (1, 9228)

data = {
    "node_feats": node_feats,
    "node_class": node_class,
    "edge": edge,
    "node_list": node_list,
    "edge_list": edge_list
}

In [5]:
np.savez_compressed("citeseer.npz", **data)

In [6]:
citeseer = np.load("citeseer.npz", allow_pickle=True)
citeseer.files

['node_feats', 'node_class', 'edge', 'node_list', 'edge_list']

In [7]:
!du citeseer.npz -h

208K	citeseer.npz


## Task

In [10]:
train_set = graph.ndata["train_mask"].nonzero().squeeze().numpy()
val_set = graph.ndata["val_mask"].nonzero().squeeze().numpy()
test_set = graph.ndata["test_mask"].nonzero().squeeze().numpy()

In [11]:
task_data = {
    "train": train_set,
    "val": val_set,
    "test": test_set
}
np.savez_compressed("citeseer_task.npz", **task_data)
